In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from patsy import dmatrices

# funciones desarrolladas
from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.impresion import *

ImportError: DLL load failed while importing _datadir: No se puede encontrar el módulo especificado.

In [4]:
dd_deptos = pd.read_csv('tablas/dd_deptos.csv')

# agrega dummies apra deptos
dd_deptos = pd.get_dummies(dd_deptos, prefix='dep', columns=['nom_depto_orig'])
dd_deptos.head()

,cod,depto_origen,depto_destino,personas_mig,pob_origen,pob_destino,dist,pbi_origen,pbi_porcen_ori,pbi_destino,...,dep_MONTEVIDEO,dep_PAYSANDU,dep_RIO NEGRO,dep_RIVERA,dep_ROCHA,dep_SALTO,dep_SAN JOSE,dep_SORIANO,dep_TACUAREMBO,dep_TREINTA Y TRES
0,102,1,2,914,1318755,73377,610726,465848031,"50,3",14214980,...,1,0,0,0,0,0,0,0,0,0
1,103,1,3,33127,1318755,520173,22544,465848031,"50,3",86306492,...,1,0,0,0,0,0,0,0,0,0
2,104,1,4,1387,1318755,84698,393703,465848031,"50,3",17048887,...,1,0,0,0,0,0,0,0,0,0
3,105,1,5,2100,1318755,123203,172773,465848031,"50,3",43788749,...,1,0,0,0,0,0,0,0,0,0
4,106,1,6,982,1318755,57084,198970,465848031,"50,3",13023942,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
y, X = dmatrices('admit ~ gre + gpa + C(rank)', df, return_type = 'dataframe')

In [89]:
X_vars = [
'dep_ARTIGAS', 'dep_CANELONES',
'dep_CERRO LARGO', 'dep_COLONIA', 'dep_DURAZNO', 'dep_FLORES',
'dep_FLORIDA', 'dep_LAVALLEJA', 'dep_MALDONADO', 'dep_MONTEVIDEO',
'dep_PAYSANDU', 'dep_RIO NEGRO', 'dep_RIVERA', 'dep_ROCHA', 'dep_SALTO',
'dep_SAN JOSE', 'dep_SORIANO', 'dep_TACUAREMBO', 'dep_TREINTA Y TRES',
'log_pbi_destino', 'log_dist']

X = dd_deptos[X_vars]

y = dd_deptos.personas_mig

#X_train, X_test, y_train, y_test = train_test_split(X, y)

In [90]:
clf = linear_model.PoissonRegressor(max_iter=10000, fit_intercept=True)

clf.fit(X, y)

PoissonRegressor(max_iter=10000)

In [91]:
print(clf.coef_)

[ 0.55402549  0.07938718  0.20408798 -0.12496685 -0.35175311 -1.12695925
 -0.58896812 -0.5859174  -0.06707649  2.25373551  0.18988379 -0.28444447
  0.37859708 -0.2300682   0.43517179 -0.72851825 -0.04970222  0.33313783
 -0.28962751  0.84598091 -0.78630861]


In [92]:
print(clf.score(X, y))
print(clf.intercept_)
print(clf.get_params())

0.9427227371147557
0.4589134114155493
{'alpha': 1.0, 'fit_intercept': True, 'max_iter': 10000, 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [93]:
#clf.predict(X_test)